In [1]:
import pandas as pd
import numpy as np
from IPython.display import HTML
from tqdm import tqdm
from Utils import *
import torch
from scipy.spatial.distance import cosine
from collections import OrderedDict



## Load and preprocess data

In [4]:
train_df = pd.read_excel('./Data/training.xls')
train_df

,file_name,misogynous,Text Transcription
0,5827.jpg,0,"*Gets on Pornhub* Woman on Porn Ad: ""Are you s..."
1,2454.jpg,0,When your high school girlfriend finally turns...
2,6492.jpg,1,Me every time I refuse to objectify women I ca...
3,2054.jpg,1,Verizon Q Search News r/kotakuinaction2 There ...
4,5388.jpg,0,me watching a horror movie NO DUDE WASN'T ME s...
...,...,...,...
9995,6179.jpg,1,LMAObama.com GET UP OFFA THAT THING
9996,11354.jpg,1,QUO 74 BUCKEYES THE OHIO STATE Where the hotti...
9997,3073.jpg,1,FEEL LIKE Doin' Some Plowin? memecenter.com Me...
9998,13096.jpg,1,BLOND WONDERING WHY SHE ONLY HAS THREE SISTERS...


In [5]:
label_df_path = "./Data/clarifai_train.csv"
train_df = train_df.merge(pd.read_csv(label_df_path), left_on='file_name', right_on='id').drop_duplicates().reset_index()
train_df

,index,file_name,misogynous,Text Transcription,id,clarifai
0,0,5827.jpg,0,"*Gets on Pornhub* Woman on Porn Ad: ""Are you s...",5827.jpg,"['cartoon', 'child', 'man']"
1,1,2454.jpg,0,When your high school girlfriend finally turns...,2454.jpg,"['cartoon', 'man', 'woman']"
2,2,6492.jpg,1,Me every time I refuse to objectify women I ca...,6492.jpg,['woman']
3,3,2054.jpg,1,Verizon Q Search News r/kotakuinaction2 There ...,2054.jpg,"['cartoon', 'man', 'woman']"
4,4,5388.jpg,0,me watching a horror movie NO DUDE WASN'T ME s...,5388.jpg,"['animal', 'cartoon']"
...,...,...,...,...,...,...
9995,9995,6179.jpg,1,LMAObama.com GET UP OFFA THAT THING,6179.jpg,['woman']
9996,9996,11354.jpg,1,QUO 74 BUCKEYES THE OHIO STATE Where the hotti...,11354.jpg,['woman']
9997,9997,3073.jpg,1,FEEL LIKE Doin' Some Plowin? memecenter.com Me...,3073.jpg,"['woman', 'nudity']"
9998,9998,13096.jpg,1,BLOND WONDERING WHY SHE ONLY HAS THREE SISTERS...,13096.jpg,['woman']


In [6]:
label_df_path = "./Data/all_misogyny.xlsx"
train_df = train_df.merge(pd.read_excel(label_df_path)[['meme', 'NOTmisogynous']], left_on='file_name', right_on='meme').drop_duplicates().reset_index()

In [7]:
train_df = get_dataset_labels(train_df, ['original_text','clarifai','misogynous','soft_label_0','soft_label_1', 'disagreement'])

In [8]:
# Convert entire column to a list (saved as str)
from ast import literal_eval
train_df['clarifai'] = train_df['clarifai'].apply(lambda x: literal_eval(x))

## Compute Scores

In [26]:
# ESCLUDO GLI ULTIMI 1000 DA USARE COME TEST
test_df = train_df[9000:]
train_df = train_df[:9000]
test_df

,original_text,clarifai,misogynous,soft_label_0,soft_label_1,disagreement,tokens,animal,broom,car,...,cat,dog,child,crockery,dishwasher,kitchen,kitchenutensil,man,woman,nudity
9000,BestDemotivationalPosters.com GAMER GIRLS Yes ...,[woman],1,0.000000,1.000000,1,[woman],0,0,0,...,0,0,0,0,0,0,0,0,1,0
9001,Hooker: $100 whatever you want baby. Me: Get i...,"[car, man, woman]",1,0.000000,1.000000,1,"[car, man, woman]",0,0,1,...,0,0,0,0,0,0,0,1,1,0
9002,DEMOCRATS HAVE DANGEROUSLY AND INTENTIONALLY M...,"[cartoon, woman]",0,0.666667,0.333333,0,"[cartoon, woman]",0,0,0,...,0,0,0,0,0,0,0,0,1,0
9003,"Yoy wanted to speak with me boss? Don't worry,...","[cartoon, kitchen]",0,1.000000,0.000000,1,"[cartoon, kitchen]",0,0,0,...,0,0,0,0,0,1,0,0,0,0
9004,"Doctor, what happens after we die? We clean th...","[man, woman]",0,1.000000,0.000000,1,"[man, woman]",0,0,0,...,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,LMAObama.com GET UP OFFA THAT THING,[woman],1,0.000000,1.000000,1,[woman],0,0,0,...,0,0,0,0,0,0,0,0,1,0
9996,QUO 74 BUCKEYES THE OHIO STATE Where the hotti...,[woman],1,0.000000,1.000000,1,[woman],0,0,0,...,0,0,0,0,0,0,0,0,1,0
9997,FEEL LIKE Doin' Some Plowin? memecenter.com Me...,"[woman, nudity]",1,0.333333,0.666667,0,"[woman, nudity]",0,0,0,...,0,0,0,0,0,0,0,0,1,1
9998,BLOND WONDERING WHY SHE ONLY HAS THREE SISTERS...,[woman],1,0.000000,1.000000,1,[woman],0,0,0,...,0,0,0,0,0,0,0,0,1,0


## Load preprocessed data

## Token selection

In [15]:
valid_tokens = ['Animal', 'Broom', 'Car', 'Cartoon', 'Cat', 'Dog', 'Child', 'Crockery', 'Dishwasher', 'Kitchen', 'KitchenUtensil', 'Man', 'Woman', 'Nudity']
valid_tokens = [str(element).lower() for element in valid_tokens]
valid_tokens

['animal',
 'broom',
 'car',
 'cartoon',
 'cat',
 'dog',
 'child',
 'crockery',
 'dishwasher',
 'kitchen',
 'kitchenutensil',
 'man',
 'woman',
 'nudity']

In [13]:
train_df['tokens']=train_df['clarifai']

In [16]:
# Create a new column for each unique token, with values of 1 or 0 depending on whether the token is in the "tokens" list for that row
train_df['tokens'] = train_df['tokens'].apply(lambda x: [str(element).lower() for element in x])
train_df = tokens_to_columns(valid_tokens, train_df).copy()

## Compute scores

In [32]:
plot_scores = pd.DataFrame(columns=['token', 'Agreement', 'Hate'])
plot_scores

,token,Agreement,Hate


In [33]:
agreement_df = pd.concat([train_df.loc[train_df['soft_label_1']==1], train_df.loc[train_df['soft_label_0']==1]])
for x in valid_tokens:
  #compute p(Agreement|t)

  #if there is only one value it's 0
  if len(agreement_df[x].value_counts()) == 1 and 0 in list(agreement_df[x].values):
    p1=0
  else:
    p1 = agreement_df[x].value_counts()[1]/train_df[x].value_counts()[1]

  #compute p(Hate|t)
  #if there is only one value it's 0
  if len(train_df.loc[train_df['misogynous']==1][x].value_counts()) == 1 and 0 in list(train_df.loc[train_df['misogynous']==1][x].values):
    p2=0
  else:
    p2 = train_df.loc[train_df['misogynous']==1][x].value_counts()[1]/train_df[x].value_counts()[1]

  #plot_scores=plot_scores.append({'token':x, 'Agreement':p1, 'Hate':p2 },ignore_index=True)
  plot_scores = pd.concat([plot_scores, pd.DataFrame([{'token': x, 'Agreement': p1, 'Hate': p2}])], ignore_index=True)


In [34]:
plot_scores

,token,Agreement,Hate
0,animal,0.629348,0.279348
1,broom,0.571429,0.857143
2,car,0.712991,0.513595
3,cartoon,0.637157,0.381546
4,cat,0.730769,0.217949
5,dog,0.597990,0.286432
6,child,0.613406,0.450914
7,crockery,0.746835,0.367089
8,dishwasher,0.500000,1.000000
9,kitchen,0.701299,0.454545


In [35]:
plot_scores['Agreement_coordinate'] = plot_scores['Agreement'].apply(lambda x: x-(1-x))
plot_scores['Hate_coordinate'] = plot_scores['Hate'].apply(lambda x: x-(1-x))

#plot_scores['Agreement_valid'] = plot_scores['Agreement_coordinate'].apply(lambda x: np.abs(x)>=0.5) #if absolute value is grather than 0.5
#plot_scores['Hate_valid'] = plot_scores['Hate_coordinate'].apply(lambda x: np.abs(x)>=0.5)
plot_scores['occurrences'] = plot_scores['token'].apply(lambda x: train_df[x].value_counts()[1])

In [37]:
tokens_df_10 = plot_scores[plot_scores.occurrences >= 10]

In [38]:
tokens_df_10.sort_values(by=['Agreement_coordinate']).to_csv('tokens_df_10_sorted.csv', sep='\t', index=False)

In [42]:
tokens_df_10.sort_values(by=['Agreement_coordinate']).head(20)

,token,Agreement,Hate,Agreement_coordinate,Hate_coordinate,occurrences
5,dog,0.597990,0.286432,0.195980,-0.427136,199
6,child,0.613406,0.450914,0.226811,-0.098172,1477
12,woman,0.627665,0.662882,0.255329,0.325765,4316
0,animal,0.629348,0.279348,0.258696,-0.441304,920
3,cartoon,0.637157,0.381546,0.274314,-0.236908,3208
11,man,0.650108,0.433045,0.300216,-0.133909,4630
9,kitchen,0.701299,0.454545,0.402597,-0.090909,77
10,kitchenutensil,0.702703,0.378378,0.405405,-0.243243,37
2,car,0.712991,0.513595,0.425982,0.027190,331
4,cat,0.730769,0.217949,0.461538,-0.564103,156


In [43]:
plot_scores.to_csv('final_scores_tags_trainOnly.csv', sep='\t', index=False)

In [9]:
plot_scores = pd.read_csv('final_scores_tags_trainOnly.csv', sep='\t')
plot_scores

,token,Agreement,Hate,Agreement_coordinate,Hate_coordinate,occurrences
0,animal,0.629348,0.279348,0.258696,-0.441304,920
1,broom,0.571429,0.857143,0.142857,0.714286,7
2,car,0.712991,0.513595,0.425982,0.027190,331
3,cartoon,0.637157,0.381546,0.274314,-0.236908,3208
4,cat,0.730769,0.217949,0.461538,-0.564103,156
5,dog,0.597990,0.286432,0.195980,-0.427136,199
6,child,0.613406,0.450914,0.226811,-0.098172,1477
7,crockery,0.746835,0.367089,0.493671,-0.265823,79
8,dishwasher,0.500000,1.000000,0.000000,1.000000,6
9,kitchen,0.701299,0.454545,0.402597,-0.090909,77


## Baseline

In [10]:
tokens_df_10 = plot_scores[plot_scores.occurrences >= 10]
tokens_df = plot_scores

### Threshold estimation from Dev

#### token at least 10 occurrences

In [27]:
dev_df = train_df[8000:]

### Baseline NO-Estimation

In [28]:
pred_somma = []
pred_tutti_verdi = []
pred_media = []
pred_mediana = []

for _, row in tqdm(dev_df.iterrows()):
  colors_agreement, _ = get_all_colors(row['tokens'], tokens_df_10)

  if 'NA' in colors_agreement:
    indexes = find_NA_indices(colors_agreement)
    for new_word_index in range(len(indexes)) :
      #stimated_coordinate = new_tokens_df_dev.loc[new_tokens_df_dev['new_token']== list(row['tokens_list'])[indexes[new_word_index]],'coordinate'].values[0]
      colors_agreement[indexes[new_word_index]]=0

  #tolgo gli zero:
  colors_agreement = [i for i in colors_agreement if i != 0]

  if colors_agreement:

    pred_somma.append(sum(colors_agreement))
    pred_media.append(np.mean(colors_agreement))
    pred_mediana.append(np.median(colors_agreement))
    pred_tutti_verdi.append(min(colors_agreement))

  else:
    pred_somma.append(0)
    pred_media.append(0)
    pred_mediana.append(0)
    pred_tutti_verdi.append(0)

1000it [00:00, 1701.49it/s]


In [29]:
threshold_estimation(pred_somma, dev_df, 'disagreement', True)

THRESHOLD: 0.3

              precision    recall  f1-score   support

           0       0.44      0.36      0.40       364
           1       0.67      0.74      0.70       636

    accuracy                           0.60      1000
   macro avg       0.56      0.55      0.55      1000
weighted avg       0.59      0.60      0.59      1000



(0.3, 0.5498803629385705)

In [30]:
threshold_estimation(pred_media, dev_df, 'disagreement', True)

THRESHOLD: 0.3

              precision    recall  f1-score   support

           0       0.40      0.81      0.54       364
           1       0.74      0.31      0.43       636

    accuracy                           0.49      1000
   macro avg       0.57      0.56      0.48      1000
weighted avg       0.61      0.49      0.47      1000



(0.3, 0.4839421166856359)

In [31]:
threshold_estimation(pred_mediana, dev_df, 'disagreement', True)

THRESHOLD: 0.3

              precision    recall  f1-score   support

           0       0.39      0.81      0.53       364
           1       0.73      0.29      0.41       636

    accuracy                           0.48      1000
   macro avg       0.56      0.55      0.47      1000
weighted avg       0.60      0.48      0.45      1000



(0.3, 0.4702354362859351)

In [32]:
threshold_estimation(pred_tutti_verdi, dev_df, 'disagreement', True)

THRESHOLD: 0.3

              precision    recall  f1-score   support

           0       0.38      0.85      0.53       364
           1       0.72      0.22      0.34       636

    accuracy                           0.45      1000
   macro avg       0.55      0.54      0.44      1000
weighted avg       0.60      0.45      0.41      1000



(0.3, 0.4356351674333736)

## Performances on unseen Test

In [33]:
test_df['tokens'] = test_df['clarifai']

In [34]:
## BASELINE
pred_somma = []
pred_tutti_verdi = []
pred_media = []
pred_mediana = []

for _, row in tqdm(test_df.iterrows()):
  colors_agreement, _ = get_all_colors(row['tokens'], tokens_df_10)

  if 'NA' in colors_agreement:
    indexes = find_NA_indices(colors_agreement)
    for new_word_index in range(len(indexes)) :
      #stimated_coordinate = new_tokens_df_dev.loc[new_tokens_df_dev['new_token']== list(row['tokens_list'])[indexes[new_word_index]],'coordinate'].values[0]
      colors_agreement[indexes[new_word_index]]=0

  #tolgo gli zero:
  colors_agreement = [i for i in colors_agreement if i != 0]

  if colors_agreement:

    pred_somma.append(sum(colors_agreement))
    pred_media.append(np.mean(colors_agreement))
    pred_mediana.append(np.median(colors_agreement))
    pred_tutti_verdi.append(min(colors_agreement))

  else:
    pred_somma.append(0)
    pred_media.append(0)
    pred_mediana.append(0)
    pred_tutti_verdi.append(0)

1000it [00:00, 2110.23it/s]


In [35]:
best_t = 0.3

print('SOMMA \n')
print('THRESHOLD: '+ str(best_t) + '\n')
print(classification_report(test_df['disagreement'], [int(i>=best_t) for i in pred_somma] ))


SOMMA 

THRESHOLD: 0.3

              precision    recall  f1-score   support

           0       0.36      0.33      0.34       332
           1       0.68      0.70      0.69       668

    accuracy                           0.58      1000
   macro avg       0.52      0.52      0.52      1000
weighted avg       0.57      0.58      0.57      1000



In [36]:
best_t = 0.3
print('MEDIA \n')
print('THRESHOLD: '+ str(best_t) + '\n')
print(classification_report(test_df['disagreement'], [int(i>=best_t) for i in pred_media] ))

MEDIA 

THRESHOLD: 0.3

              precision    recall  f1-score   support

           0       0.35      0.78      0.48       332
           1       0.72      0.28      0.41       668

    accuracy                           0.45      1000
   macro avg       0.54      0.53      0.45      1000
weighted avg       0.60      0.45      0.43      1000



In [37]:
best_t = 0.3
print('MEDIANA \n')
print('THRESHOLD: '+ str(best_t) + '\n')
print(classification_report(test_df['disagreement'], [int(i>=best_t) for i in pred_mediana] ))

MEDIANA 

THRESHOLD: 0.3

              precision    recall  f1-score   support

           0       0.36      0.80      0.49       332
           1       0.74      0.28      0.41       668

    accuracy                           0.45      1000
   macro avg       0.55      0.54      0.45      1000
weighted avg       0.61      0.45      0.43      1000



In [38]:
best_t = 0.3

print(' ALL GREEN \n')
print('THRESHOLD: '+ str(best_t) + '\n')
print(classification_report(test_df['disagreement'], [int(i>=best_t) for i in pred_tutti_verdi] ))

 ALL GREEN 

THRESHOLD: 0.3

              precision    recall  f1-score   support

           0       0.35      0.83      0.49       332
           1       0.73      0.23      0.35       668

    accuracy                           0.43      1000
   macro avg       0.54      0.53      0.42      1000
weighted avg       0.61      0.43      0.40      1000

